In [ ]:
import tensorflow as tf 
tf.test.gpu_device_name() 

'/device:GPU:0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%cd "/content/drive/My Drive/mask_type_class"

/content/drive/My Drive/mask_type_class


In [ ]:
#!unzip type_dataset.zip

In [ ]:
import os,shutil
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
import json

In [ ]:
from skimage.io import imread, imshow, imsave

In [ ]:
import random

# Data Preprocessing

In [ ]:
path_to_annotations = '/Users/shruthiravi/Documents/rsi_20/mask_data/annotations/'
list_of_annotations = os.listdir(path = path_to_annotations)

In [ ]:
correct_files = []
for number in list_of_annotations:
    if '.json' in number:
        correct_files.append(number)
    else:
        print(number)

In [ ]:
all_annotations = {}
for fname in correct_files:
    # open file     
    with open(os.path.join(path_to_annotations, fname)) as f:
        # load file & get into dict
        data = json.load(f)    
    # extract value of the FileName key
    fileNameKey = data['FileName']
    # initialize empty list
    annotations_list = []
    # for each element in annotations
    for element in data['Annotations']:
        # extract bounding_box
        bounding_box = element['BoundingBox']
        # extract class_name
        class_name = element['classname']
        # put them into a tuple 
        final_annotation = (bounding_box, class_name)
        # put the tuple into list
        annotations_list.append(final_annotation)
    # list becomes value of key
    all_annotations[fileNameKey] = annotations_list

In [ ]:
print(all_annotations)

In [ ]:
classes = []
for key in all_annotations:
    for element in all_annotations[key]:
        classes.append(element[1])
set_of_classes = set(classes)

In [ ]:
set([e[1] for k in all_annotations.keys() for e in all_annotations[k] ])

In [ ]:
from collections import Counter

In [ ]:
Counter(classes)

In [ ]:
base_dir = '/Users/shruthiravi/Documents/rsi_20/mask_type_classification'

In [ ]:
splits = [x for x in os.listdir(base_dir) if x != '.DS_Store']
splits

In [ ]:
folds = ['train', 'test', 'valid']

In [ ]:
mask_types = ['mask_surgical','scarf_bandana', 'mask_colorful','face_shield']

In [ ]:
for fold in folds:
     os.mkdir(os.path.join(base_dir, fold))

In [ ]:
for fold in folds:
    for folder in mask_types:
        splits_dir = os.path.join(base_dir, fold)
        os.mkdir(os.path.join(splits_dir, folder))

In [ ]:
def save_photos(tuple_of_info, path_to_save, occurence, img_path='/Users/shruthiravi/Documents/rsi_20/mask_data/images'):
    '''this function takes in a tuple'''
    name, label, bb = tuple_of_info
    final_path = os.path.join(path_to_save, label)
    image = imread(os.path.join(img_path, name))
    [tl_x, tl_y, br_x, br_y] = bb
    cropped = image[tl_y:br_y, tl_x:br_x, :] 
    name_pieces = name.split('.')
    new_name = name_pieces[0] + f"_{occurence}." + name_pieces[1]
    further_path = os.path.join(path_to_save, label)
    imsave(os.path.join(further_path, new_name), cropped)

In [ ]:
#current_img_name = ''
needed_labels = ['face_with_mask', 'face_other_covering','face_with_mask_incorrect']
mask_labels = ['mask_surgical','scarf_bandana', 'mask_colorful','face_shield']
cropped_annotations = [] # list of three-tuples, each containing (img_name, label, bb)
#test = []
#train = []
#valid = []
# for annotation in annotations_list
for current_img_name in all_annotations.keys():
    for i, annotation in enumerate(all_annotations[current_img_name]):
        # extract bounding box
        bb = annotation[0]
        # crop to bounding box
        # extract classname
        label = annotation[1]
        # crop to bounding box    
        if label in needed_labels: 
            [tl_x, tl_y, br_x, br_y] = bb
            if (br_x-tl_x) > 0 and (br_y-tl_y) >0:
                for val_label in all_annotations[current_img_name][i+1:]:
                    mask_type = val_label[1]
                    if mask_type in mask_labels:
                        specific_bb = val_label[0]
                        [tl_c_x, tl_c_y,br_c_x,br_c_y] = specific_bb
                        if tl_c_x > tl_x & tl_c_y > tl_y & br_c_x < br_x & br_c_y < br_y:
                            fixed = (current_img_name, mask_type, bb)
                            cropped_annotations.append(fixed) 

In [ ]:
#save_photos(train[1301],'/Users/shruthiravi/Documents/rsi_20/mask_type_classification/train', 'test')

In [ ]:
# randomly shuffle the list 
random.shuffle(cropped_annotations)
# determine which split
total_size = len(cropped_annotations)
perc_train = 0.7
perc_test = 0.15
perc_valid = 0.15

train = cropped_annotations[0:int(perc_train*total_size)]
test = cropped_annotations[int(perc_train*total_size):int((perc_train+perc_test)*total_size)]
valid = cropped_annotations[int((perc_train+perc_test)*total_size):]
fold_dict = {'train': train, 'test': test, 'valid': valid}

In [ ]:
Counter([b[1] for b in train])

In [ ]:
dest_path = '/Users/shruthiravi/Documents/rsi_20/mask_type_classification/'
# train_path = '/Users/shruthiravi/Documents/rsi_20/mask_type_classification/train'
# test_path = '/Users/shruthiravi/Documents/rsi_20/mask_type_classification/test'
# valid_path = '/Users/shruthiravi/Documents/rsi_20/mask_type_classification/valid'
num_occ = {}

for fold in fold_dict:
    for example in fold_dict[fold]:
        if example[0] in num_occ:
            num_occ[example[0]] += 1
        else:
            num_occ[example[0]] = 0
        save_photos(example, os.path.join(dest_path, fold), num_occ[example[0]])
        print(example)

In [ ]:
def print_img(fname):
    test_image = imread('/Users/shruthiravi/Documents/rsi_20/mask_data/images/%s' % fname)
    test_annotations = all_annotations[fname]
    y, x, c = test_image.shape
    colors = ['red', 'green', 'blue', 'pink', 'black']
    for i, ann in enumerate(test_annotations):
        [tl_x, tl_y, br_x, br_y] = ann[0]
        color = colors[i%len(colors)]
        plt.plot(range(tl_x, br_x),[tl_y for _ in range(tl_x, br_x)],  c=color, label = ann[1])
        plt.plot([tl_x for _ in range(tl_y, br_y)],range(tl_y, br_y),  c=color)
        plt.plot(range(tl_x, br_x), [br_y for _ in range(tl_x, br_x)], c=color)
        plt.plot([br_x for _ in range(tl_y, br_y)], range(tl_y, br_y), c=color)
        #plt.legend()
        imshow(test_image)

In [ ]:
!ls {train_dir}

ls: cannot access '/content/drive/My': No such file or directory
ls: cannot access 'Drive/mask_type_class/updated/train/': No such file or directory


In [ ]:
!ls {test_dir}

mask_colorful  mask_surgical  respirator  scarf_bandana


In [ ]:
!ls {val_dir}

cloth  respirator  scarf_bandana  surgical


# Actual Model

In [ ]:
base_path = r"""/content/drive/My Drive/mask_type_class/updated/"""

In [ ]:
train_dir = base_path + "train/"

In [ ]:
test_dir = base_path + "test/"

In [ ]:
val_dir = base_path + "valid/"

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=20,
    class_mode='categorical')

Found 4100 images belonging to 4 classes.


In [ ]:
validation_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(224,224),
    batch_size=20,
    class_mode='categorical')

Found 897 images belonging to 4 classes.


In [ ]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 224, 224, 3)
labels batch shape: (20, 4)


# Define the Model

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation = 'relu',
                       input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(4,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 128)      

# Compile 

In [ ]:
from keras import optimizers

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

# Fit using Batch Generator

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=40,
    validation_data=validation_generator,
    validation_steps=20)

Epoch 1/40
100/100 [==============================] - 16s 164ms/step - loss: 0.8988 - acc: 0.6785 - val_loss: 0.5427 - val_acc: 0.7525
Epoch 2/40
100/100 [==============================] - 10s 104ms/step - loss: 0.8195 - acc: 0.7320 - val_loss: 0.7805 - val_acc: 0.7550
Epoch 3/40
100/100 [==============================] - 11s 112ms/step - loss: 0.7518 - acc: 0.7515 - val_loss: 0.7136 - val_acc: 0.7431
Epoch 4/40
100/100 [==============================] - 10s 103ms/step - loss: 0.7530 - acc: 0.7465 - val_loss: 0.6623 - val_acc: 0.7475
Epoch 5/40
100/100 [==============================] - 11s 110ms/step - loss: 0.7120 - acc: 0.7540 - val_loss: 1.0393 - val_acc: 0.7632
Epoch 6/40
100/100 [==============================] - 10s 101ms/step - loss: 0.6746 - acc: 0.7735 - val_loss: 0.6285 - val_acc: 0.7800
Epoch 7/40
100/100 [==============================] - 10s 101ms/step - loss: 0.7104 - acc: 0.7525 - val_loss: 0.6337 - val_acc: 0.7809
Epoch 8/40
100/100 [==============================] - 1

# Save The Model

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save("updated_types_colab.h5")
print("Saved model to disk")

Saved model to disk


# Test it Out

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model


model = load_model('my_model_1.h5')

# Load model
#model = load_model('model.json')

#test_datagen = ImageDataGenerator(rescale=1./255)
#test_generator = test_datagen.flow_from_directory(
#        test_dir,
#        target_size=(150, 150),
#        batch_size=20,
#        class_mode='categorical',
#        shuffle=False)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(28, 28),
    color_mode="rgb",
    batch_size=32,
    class_mode=None,
    shuffle=False
)

In [ ]:
test_generator.reset()

In [ ]:
pred=cnn.predict_generator(test_generator,verbose=1,steps=306/batch_size)

In [ ]:
model.summary()

# Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
from keras.preprocessing import image

In [ ]:
train_face_shield_dir = '/Users/shruthiravi/Documents/rsi_20/mask_type_classification/train/face_shield/'

In [ ]:
fnames = [os.path.join(train_face_shield_dir, fname) for
         fname in os.listdir(train_face_shield_dir)]

In [ ]:
img_path = fnames[3]

In [ ]:
img = image.load_img(img_path, target_size=(150,150))

In [ ]:
x = image.img_to_array(img)

In [ ]:
x = x.reshape((1,) + x.shape)

In [ ]:
i = 0
for batch in datagen.flow(x,batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0 :
        break

In [ ]:
plt.show()

# Adding Dropout

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation = 'relu',
                       input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(4,activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

In [ ]:
train_datagen = ImageDataGenerator (
    rescale=1./255,
    rotation_range=40,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=-.2,
    horizontal_flip=True,)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='categorical')

Found 4100 images belonging to 4 classes.


In [ ]:
validation_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='categorical')

Found 897 images belonging to 4 classes.


In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 15, 15, 128)      

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=40,
    validation_data=validation_generator,
    validation_steps=30)

Epoch 1/40
100/100 [==============================] - 24s 238ms/step - loss: 0.9408 - acc: 0.6575 - val_loss: 0.9655 - val_acc: 0.7150
Epoch 2/40
100/100 [==============================] - 22s 216ms/step - loss: 0.8634 - acc: 0.6885 - val_loss: 0.8638 - val_acc: 0.7270
Epoch 3/40
100/100 [==============================] - 22s 222ms/step - loss: 0.8094 - acc: 0.7110 - val_loss: 0.4737 - val_acc: 0.7839
Epoch 4/40
100/100 [==============================] - 22s 221ms/step - loss: 0.8292 - acc: 0.7055 - val_loss: 0.9014 - val_acc: 0.7483
Epoch 5/40
100/100 [==============================] - 22s 221ms/step - loss: 0.7930 - acc: 0.7150 - val_loss: 1.3334 - val_acc: 0.7722
Epoch 6/40
100/100 [==============================] - 22s 222ms/step - loss: 0.7666 - acc: 0.7405 - val_loss: 0.8683 - val_acc: 0.7789
Epoch 7/40
100/100 [==============================] - 22s 221ms/step - loss: 0.7838 - acc: 0.7270 - val_loss: 0.6976 - val_acc: 0.7617
Epoch 8/40
100/100 [==============================] - 2

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("dropout_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save("updated_types_dropout_7_20.h5")
print("Saved model to disk")

Saved model to disk


# NEW DATA 

In [ ]:
train_dir = '/Users/shruthiravi/Documents/rsi_20/combined/train/'

In [ ]:
test_dir = '/Users/shruthiravi/Documents/rsi_20/combined/test/'

In [ ]:
val_dir = '/Users/shruthiravi/Documents/rsi_20/combined/valid/'

# Actual Model

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='categorical')

Found 4100 images belonging to 4 classes.


In [ ]:
validation_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='categorical')

Found 897 images belonging to 4 classes.


In [ ]:
i = 0
for batch in validation_generator.:
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0 :
        break

In [ ]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20, 4)


# Define the Model

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation = 'relu',
                       input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(4,activation='softmax'))

Using TensorFlow backend.


KeyboardInterrupt: ignored

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

# Compile 

In [ ]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

# Fit using Batch Generator

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochst=20,
    validation_data=validation_generator,
    validation_steps=20)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
100/100 [==============================] - 52s 522ms/step - loss: 0.3875 - acc: 0.6795 - val_loss: 0.3189 - val_acc: 0.7700
Epoch 2/20
100/100 [==============================] - 54s 543ms/step - loss: 0.3427 - acc: 0.7400 - val_loss: 0.3323 - val_acc: 0.7550
Epoch 3/20
100/100 [==============================] - 53s 529ms/step - loss: 0.3359 - acc: 0.7455 - val_loss: 0.3179 - val_acc: 0.7500
Epoch 4/20
100/100 [==============================] - 51s 508ms/step - loss: 0.3251 - acc: 0.7535 - val_loss: 0.3111 - val_acc: 0.7700
Epoch 5/20
100/100 [==============================] - 51s 510ms/step - loss: 0.3097 - acc: 0.7530 - val_loss: 0.2868 - val_acc: 0.7850
Epoch 6/20
100/100 [==============================] - 53s 529ms/step - loss: 0.2990 - acc: 0.7710 - val_loss: 0.3039 - val_acc: 0.7700
Epoch 7/20
100/100 [==============================] - 50s 503ms/step - loss: 0.3067 - acc: 0.7490 - val_loss: 0.35

# Load the Pretrained Model

In [ ]:
# example of loading the resnet50 model
from keras.applications.resnet50 import ResNet50

In [ ]:
from keras.layers import Dense

In [ ]:
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import Callback

In [ ]:
help(ResNet50)

Help on function wrapper in module keras.applications:

wrapper(*args, **kwargs)



In [ ]:
# load model
resnet_model = ResNet50(
                classes = 4,
                pooling = 'max',
                weights = 'imagenet',
                include_top = False)

94658560/94653016 [==============================] - 2s 0us/step


In [ ]:
# add a global spatial average pooling layer
x = resnet_model.output
# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(4, activation='softmax')(x)

In [ ]:
# summarize the model
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
___________________________________________________________________________________________

In [ ]:
weighted_model = Model(inputs=resnet_model.input, outputs=predictions)

In [ ]:
weighted_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
____________________________________________________________________________________________

In [ ]:
from keras import optimizers

weighted_model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
mc = ModelCheckpoint('best_weighted_multi_model_3.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
history = weighted_model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=1000,
    validation_data=validation_generator,
    validation_steps=40,
    callbacks=[es,mc])

Epoch 1/1000
100/100 [==============================] - 49s 490ms/step - loss: 0.0855 - acc: 0.9695 - val_loss: 31.1663 - val_acc: 0.8150

Epoch 00001: val_acc improved from -inf to 0.81500, saving model to best_weighted_multi_model_3.h5
Epoch 2/1000
100/100 [==============================] - 49s 486ms/step - loss: 0.0782 - acc: 0.9780 - val_loss: 9.4459 - val_acc: 0.8231

Epoch 00002: val_acc improved from 0.81500 to 0.82309, saving model to best_weighted_multi_model_3.h5
Epoch 3/1000
100/100 [==============================] - 49s 488ms/step - loss: 0.0800 - acc: 0.9790 - val_loss: 0.2340 - val_acc: 0.8130

Epoch 00003: val_acc did not improve from 0.82309
Epoch 4/1000
100/100 [==============================] - 49s 486ms/step - loss: 0.1016 - acc: 0.9700 - val_loss: 2.3237 - val_acc: 0.8105

Epoch 00004: val_acc did not improve from 0.82309
Epoch 5/1000
100/100 [==============================] - 49s 486ms/step - loss: 0.2386 - acc: 0.9650 - val_loss: 0.7562 - val_acc: 0.8218

Epoch 00

In [ ]:
# serialize model to JSON
weighted_model_json = weighted_model.to_json()
with open("weighted_model.json", "w") as json_file:
    json_file.write(weighted_model_json)
# serialize weights to HDF5
weighted_model.save("resnet_weighted_1.h5")
print("Saved model to disk")

Saved model to disk


# VGG19


In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
from keras.layers import Dense

In [ ]:
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import Callback

In [ ]:
# load model
vgg_model = VGG16()

553467904/553467096 [==============================] - 43s 0us/step


In [ ]:
# add a global spatial average pooling layer
x = vgg_model.output
# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(4, activation='softmax')(x)

In [ ]:
# summarize the model
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
wvgg_model = Model(inputs=vgg_model.input, outputs=predictions)

In [ ]:
wvgg_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
from keras import optimizers

wvgg_model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
mc = ModelCheckpoint('best_weighted_vgg_multi_model_3.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
history = wvgg_model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=1000,
    validation_data=validation_generator,
    validation_steps=40,
    callbacks=[es,mc])

Epoch 1/1000
100/100 [==============================] - 1916s 19s/step - loss: 1.2613 - acc: 0.6315 - val_loss: 1.2201 - val_acc: 0.6625

Epoch 00001: val_acc improved from -inf to 0.66250, saving model to best_weighted_vgg_multi_model_3.h5
Epoch 2/1000
100/100 [==============================] - 1190s 12s/step - loss: 1.1385 - acc: 0.6300 - val_loss: 1.2425 - val_acc: 0.6725

Epoch 00002: val_acc improved from 0.66250 to 0.67252, saving model to best_weighted_vgg_multi_model_3.h5
Epoch 3/1000
100/100 [==============================] - 141s 1s/step - loss: 1.0248 - acc: 0.6500 - val_loss: 1.2019 - val_acc: 0.6625

Epoch 00003: val_acc did not improve from 0.67252
Epoch 4/1000
100/100 [==============================] - 85s 853ms/step - loss: 1.0022 - acc: 0.6245 - val_loss: 0.9479 - val_acc: 0.6487

Epoch 00004: val_acc did not improve from 0.67252
Epoch 5/1000
100/100 [==============================] - 85s 852ms/step - loss: 0.9623 - acc: 0.6405 - val_loss: 1.0814 - val_acc: 0.6600

Epo

OSError: ignored

In [ ]:
# serialize model to JSON
weighted_model_json = weighted_model.to_json()
with open("weighted_model.json", "w") as json_file:
    json_file.write(weighted_model_json)
# serialize weights to HDF5
weighted_model.save("resnet_weighted_1.h5")
print("Saved model to disk")

Saved model to disk
